In [46]:
import boto3
import pandas as pd
from io import StringIO, BytesIO
from datetime import datetime, timedelta

# Inicializar el recurso S3 y el bucket
s3 = boto3.resource('s3')
bucket_name = 'xetra-1234'
bucket = s3.Bucket(bucket_name)

# Filtrar objetos por fecha
prefixes = ['2022-03-15', '2022-03-16']
objects = []
for prefix in prefixes:
    bucket_objs = bucket.objects.filter(Prefix=prefix)
    objects.extend(bucket_objs)

# Leer y concatenar los archivos CSV en un solo DataFrame
dfs = []
for obj in objects:
    csv_obj = bucket.Object(key=obj.key).get().get('Body').read().decode('utf-8')
    data = StringIO(csv_obj)
    df = pd.read_csv(data, delimiter=',')
    dfs.append(df)

df_all = pd.concat(dfs, ignore_index=True)

C:\Users\JOSUE\AppData\Local\Temp\ipykernel_16888\787105505.py:26: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_all = pd.concat(dfs, ignore_index=True)


In [47]:
print(df_all.columns)

Index(['ISIN', 'Mnemonic', 'SecurityDesc', 'SecurityType', 'Currency',
       'SecurityID', 'Date', 'Time', 'StartPrice', 'MaxPrice', 'MinPrice',
       'EndPrice', 'TradedVolume', 'NumberOfTrades'],
      dtype='object')


In [48]:
columns=['ISIN', 'Date', 'Time', 'StartPrice', 'MaxPrice', 'MinPrice','EndPrice', 'TradedVolume']
df_all = df_all.loc[:, columns]

In [49]:
df_all

,ISIN,Date,Time,StartPrice,MaxPrice,MinPrice,EndPrice,TradedVolume
0,DE000A0DJ6J9,2022-03-15,08:00,37.34,37.76,37.30,37.50,11421
1,DE000A0D6554,2022-03-15,08:00,16.58,16.58,16.43,16.45,21537
2,DE000A0D9PT0,2022-03-15,08:00,201.10,202.40,201.00,202.40,1585
3,DE000A0JL9W6,2022-03-15,08:00,62.55,63.20,62.55,63.20,1539
4,DE000A0LD6E6,2022-03-15,08:00,61.40,61.40,60.95,61.05,329
...,...,...,...,...,...,...,...,...
251797,US02005N1000,2022-03-16,20:30,37.80,37.80,37.80,37.80,0
251798,US2533931026,2022-03-16,20:30,101.00,101.00,101.00,101.00,0
251799,US89832Q1094,2022-03-16,20:30,52.50,52.50,52.50,52.50,0
251800,GB00BNYK8G86,2022-03-16,20:30,5.20,5.20,5.20,5.20,0


## Get openning price per ISIN and day

In [50]:
df_all['opening_price'] = df_all.sort_values(by=['Time']).groupby(['ISIN', 'Date'])['StartPrice'].transform('first')

In [51]:
df_all.head()

,ISIN,Date,Time,StartPrice,MaxPrice,MinPrice,EndPrice,TradedVolume,opening_price
0,DE000A0DJ6J9,2022-03-15,08:00,37.34,37.76,37.30,37.50,11421,37.34
1,DE000A0D6554,2022-03-15,08:00,16.58,16.58,16.43,16.45,21537,16.58
2,DE000A0D9PT0,2022-03-15,08:00,201.10,202.40,201.00,202.40,1585,201.10
3,DE000A0JL9W6,2022-03-15,08:00,62.55,63.20,62.55,63.20,1539,62.55
4,DE000A0LD6E6,2022-03-15,08:00,61.40,61.40,60.95,61.05,329,61.40


In [52]:
df_all[df_all['ISIN']== 'DE000A0DJ6J9']

,ISIN,Date,Time,StartPrice,MaxPrice,MinPrice,EndPrice,TradedVolume,opening_price
0,DE000A0DJ6J9,2022-03-15,08:00,37.34,37.76,37.30,37.50,11421,37.34
158,DE000A0DJ6J9,2022-03-15,08:01,37.28,37.28,37.20,37.20,2157,37.34
294,DE000A0DJ6J9,2022-03-15,08:02,37.14,37.28,37.06,37.06,2284,37.34
578,DE000A0DJ6J9,2022-03-15,08:03,37.06,37.06,36.92,36.94,1608,37.34
2999,DE000A0DJ6J9,2022-03-15,08:05,36.90,36.90,36.78,36.78,2449,37.34
...,...,...,...,...,...,...,...,...,...
246765,DE000A0DJ6J9,2022-03-16,16:25,38.14,38.14,38.10,38.10,430,38.38
247416,DE000A0DJ6J9,2022-03-16,16:27,38.14,38.14,38.08,38.08,193,38.38
247734,DE000A0DJ6J9,2022-03-16,16:28,38.10,38.10,38.06,38.10,316,38.38
248107,DE000A0DJ6J9,2022-03-16,16:29,38.08,38.18,38.08,38.12,374,38.38


## Get closing price per ISIN and day

In [53]:
df_all['Closing_Price'] = df_all.sort_values(by=['Time']).groupby(['ISIN', 'Date'])['EndPrice'].transform('last')

In [54]:
df_all

,ISIN,Date,Time,StartPrice,MaxPrice,MinPrice,EndPrice,TradedVolume,opening_price,Closing_Price
0,DE000A0DJ6J9,2022-03-15,08:00,37.34,37.76,37.30,37.50,11421,37.34,37.06
1,DE000A0D6554,2022-03-15,08:00,16.58,16.58,16.43,16.45,21537,16.58,16.40
2,DE000A0D9PT0,2022-03-15,08:00,201.10,202.40,201.00,202.40,1585,201.10,204.70
3,DE000A0JL9W6,2022-03-15,08:00,62.55,63.20,62.55,63.20,1539,62.55,61.30
4,DE000A0LD6E6,2022-03-15,08:00,61.40,61.40,60.95,61.05,329,61.40,61.45
...,...,...,...,...,...,...,...,...,...,...
251797,US02005N1000,2022-03-16,20:30,37.80,37.80,37.80,37.80,0,37.80,37.80
251798,US2533931026,2022-03-16,20:30,101.00,101.00,101.00,101.00,0,101.00,101.00
251799,US89832Q1094,2022-03-16,20:30,52.50,52.50,52.50,52.50,0,52.50,52.50
251800,GB00BNYK8G86,2022-03-16,20:30,5.20,5.20,5.20,5.20,0,5.20,5.20


In [55]:
df_all[df_all['ISIN']== 'DE000A0D9PT0']

,ISIN,Date,Time,StartPrice,MaxPrice,MinPrice,EndPrice,TradedVolume,opening_price,Closing_Price
2,DE000A0D9PT0,2022-03-15,08:00,201.1,202.4,201.0,202.4,1585,201.1,204.7
160,DE000A0D9PT0,2022-03-15,08:01,202.5,202.9,201.0,201.2,929,201.1,204.7
297,DE000A0D9PT0,2022-03-15,08:02,201.4,201.4,201.0,201.1,384,201.1,204.7
580,DE000A0D9PT0,2022-03-15,08:03,201.3,201.3,200.5,200.6,562,201.1,204.7
782,DE000A0D9PT0,2022-03-15,08:04,200.6,200.6,200.1,200.3,168,201.1,204.7
...,...,...,...,...,...,...,...,...,...,...
247096,DE000A0D9PT0,2022-03-16,16:26,213.0,213.0,212.8,213.0,952,208.9,213.0
247418,DE000A0D9PT0,2022-03-16,16:27,212.9,212.9,212.8,212.9,380,208.9,213.0
247737,DE000A0D9PT0,2022-03-16,16:28,212.9,212.9,212.8,212.8,328,208.9,213.0
248111,DE000A0D9PT0,2022-03-16,16:29,212.9,213.4,212.7,213.3,1337,208.9,213.0


## Agregations

In [56]:
df_all = df_all.groupby(['ISIN','Date'], as_index=False).agg(opening_price_eur=('opening_price', 'min'), closing_price_eur=('Closing_Price', 'min'),minimun_price_eur=('MinPrice', 'min'), maximun_price_eur=('MaxPrice', 'min'), daily_traded_volumen=('TradedVolume','sum')) 

In [57]:
df_all

,ISIN,Date,opening_price_eur,closing_price_eur,minimun_price_eur,maximun_price_eur,daily_traded_volumen
0,AT000000STR1,2022-03-15,36.4000,37.0500,36.4000,36.4000,536
1,AT000000STR1,2022-03-16,37.5000,37.1500,37.1500,37.1500,816
2,AT00000FACC2,2022-03-15,7.8800,7.7100,7.6100,7.6100,68
3,AT00000FACC2,2022-03-16,7.9500,8.0800,7.9500,7.9500,696
4,AT0000606306,2022-03-15,13.3000,13.3600,12.8800,12.9000,24002
...,...,...,...,...,...,...,...
6387,XS2427474023,2022-03-16,24.8060,24.8060,24.8060,24.8060,0
6388,XS2434891219,2022-03-15,3.4840,3.5828,3.4840,3.4840,700
6389,XS2434891219,2022-03-16,3.6134,3.6558,3.6134,3.6134,27224
6390,XS2437455608,2022-03-15,27.0020,27.0020,27.0020,27.0020,0


## percent change prev closing

In [58]:
df_all['prev_closing_price'] = df_all.sort_values(by=['Date']).groupby(['ISIN'])['closing_price_eur'].shift(1)

In [59]:
df_all

,ISIN,Date,opening_price_eur,closing_price_eur,minimun_price_eur,maximun_price_eur,daily_traded_volumen,prev_closing_price
0,AT000000STR1,2022-03-15,36.4000,37.0500,36.4000,36.4000,536,NaN
1,AT000000STR1,2022-03-16,37.5000,37.1500,37.1500,37.1500,816,37.0500
2,AT00000FACC2,2022-03-15,7.8800,7.7100,7.6100,7.6100,68,NaN
3,AT00000FACC2,2022-03-16,7.9500,8.0800,7.9500,7.9500,696,7.7100
4,AT0000606306,2022-03-15,13.3000,13.3600,12.8800,12.9000,24002,NaN
...,...,...,...,...,...,...,...,...
6387,XS2427474023,2022-03-16,24.8060,24.8060,24.8060,24.8060,0,25.0960
6388,XS2434891219,2022-03-15,3.4840,3.5828,3.4840,3.4840,700,NaN
6389,XS2434891219,2022-03-16,3.6134,3.6558,3.6134,3.6134,27224,3.5828
6390,XS2437455608,2022-03-15,27.0020,27.0020,27.0020,27.0020,0,NaN


In [60]:
df_all['change_prev_closing_%'] = (df_all['closing_price_eur'] -df_all['prev_closing_price']) / df_all ['prev_closing_price'] * 100

In [61]:
df_all.drop(columns=['prev_closing_price'], inplace=True)

In [62]:
df_all = df_all.round(decimals=2)

In [63]:
df_all

,ISIN,Date,opening_price_eur,closing_price_eur,minimun_price_eur,maximun_price_eur,daily_traded_volumen,change_prev_closing_%
0,AT000000STR1,2022-03-15,36.40,37.05,36.40,36.40,536,NaN
1,AT000000STR1,2022-03-16,37.50,37.15,37.15,37.15,816,0.27
2,AT00000FACC2,2022-03-15,7.88,7.71,7.61,7.61,68,NaN
3,AT00000FACC2,2022-03-16,7.95,8.08,7.95,7.95,696,4.80
4,AT0000606306,2022-03-15,13.30,13.36,12.88,12.90,24002,NaN
...,...,...,...,...,...,...,...,...
6387,XS2427474023,2022-03-16,24.81,24.81,24.81,24.81,0,-1.16
6388,XS2434891219,2022-03-15,3.48,3.58,3.48,3.48,700,NaN
6389,XS2434891219,2022-03-16,3.61,3.66,3.61,3.61,27224,2.04
6390,XS2437455608,2022-03-15,27.00,27.00,27.00,27.00,0,NaN


## SAVE TO S3

In [67]:
key = 'xetra_daily_report_' + datetime.today().strftime("%Y%m%d_%H%M%S") + '.parquet'

In [71]:
out_buffer = BytesIO()
df_all.to_parquet(out_buffer, index=False)
bucket_target = s3.Bucket('josh-123456')
bucket_target.put_object(Body=out_buffer.getvalue(), Key=key)

s3.Object(bucket_name='josh-123456', key='xetra_daily_report_20240325_204538.parquet')

## READING THEUPLOAD FILE

In [72]:
for obj in bucket_target.objects.all():
    print(obj.key)

xetra_daily_report_20240325_204538.parquet


In [74]:
prq_obj = bucket_target.Object(key='xetra_daily_report_20240325_204538.parquet').get().get('Body').read()
data = BytesIO(prq_obj)
df_report = pd.read_parquet(data)

In [75]:
df_report

,ISIN,Date,opening_price_eur,closing_price_eur,minimun_price_eur,maximun_price_eur,daily_traded_volumen,change_prev_closing_%
0,AT000000STR1,2022-03-15,36.40,37.05,36.40,36.40,536,NaN
1,AT000000STR1,2022-03-16,37.50,37.15,37.15,37.15,816,0.27
2,AT00000FACC2,2022-03-15,7.88,7.71,7.61,7.61,68,NaN
3,AT00000FACC2,2022-03-16,7.95,8.08,7.95,7.95,696,4.80
4,AT0000606306,2022-03-15,13.30,13.36,12.88,12.90,24002,NaN
...,...,...,...,...,...,...,...,...
6387,XS2427474023,2022-03-16,24.81,24.81,24.81,24.81,0,-1.16
6388,XS2434891219,2022-03-15,3.48,3.58,3.48,3.48,700,NaN
6389,XS2434891219,2022-03-16,3.61,3.66,3.61,3.61,27224,2.04
6390,XS2437455608,2022-03-15,27.00,27.00,27.00,27.00,0,NaN
